# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree()
            

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124057e+02     1.659169e+00
 * time: 0.7234539985656738
     1     1.105723e+01     9.008074e-01
 * time: 1.9188990592956543
     2    -1.140068e+01     1.082332e+00
 * time: 2.491284132003784
     3    -3.419012e+01     8.108837e-01
 * time: 3.3433971405029297
     4    -4.778864e+01     5.905854e-01
 * time: 4.232666015625
     5    -5.705555e+01     1.816030e-01
 * time: 5.09128999710083
     6    -5.994715e+01     7.525200e-02
 * time: 5.661306142807007
     7    -6.089002e+01     6.032427e-02
 * time: 6.220146179199219
     8    -6.130937e+01     5.891921e-02
 * time: 6.806177139282227
     9    -6.161888e+01     3.381117e-02
 * time: 7.3673670291900635
    10    -6.184759e+01     2.697570e-02
 * time: 7.942413091659546
    11    -6.198341e+01     1.899891e-02
 * time: 8.518501043319702
    12    -6.206118e+01     1.699742e-02
 * time: 9.083923101425171
    13    -6.211459e+01     1.259310e-02
 * time: 9.650257110595703
    14 

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671078
    AtomicLocal         -18.8557697
    AtomicNonlocal      14.8522648
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485383 
    Xc                  -19.3336825

    total               -62.261666460617